In [13]:
#import all libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import zipfile
import logging

In [14]:
def get_links(input_url):
    pass #TODO

In [15]:
def get_soup(input_url):
    headers = {
        'User-Agent': 'My User Agent 1.0',
        "Accept-Language": "en-US,en;q=0.5"
    }
    response = requests.get(input_url, headers=headers)
    if response.status_code != 200:
        print("Error in getting link")
        print("response code is : ", response.status_code)
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup

In [16]:
def get_fa_title(soup):
    pass #TODO

In [17]:
def get_en_title(soup):
    pass #TODO

In [18]:
def get_price(soup):
    pass #TODO

In [19]:
def get_discount(soup):
    pass #TODO

In [20]:
def get_score(soup):
    pass #TODO

In [21]:
def get_publisher(soup):
    pass #TODO

In [22]:
def get_author(soup):
    pass #TODO

In [26]:
def get_code(soup):
    soups=soup.select('tr:nth-child(1) td+ td')
    codes=[]
    for s in soups:
        codes.append(s.text)

    return codes

In [29]:
def get_isbn(soup):
    soups = soup.select('.product-table .ltr')
    isbn_numbers = []
    for inum in soups:
        cleaned_text = inum.text.strip()  
        cleaned_text = re.sub('[^0-9-]', '', cleaned_text)  
        isbn_numbers.append(cleaned_text)

    return isbn_numbers

In [ ]:
def get_pages(soup):
    pass #TODO

In [ ]:
def get_per_date(soup):
    pass #TODO

In [ ]:
def get_ad_date(soup):
    pass #TODO

In [ ]:
def get_size(soup):
    pass #TODO

In [ ]:
def get_material(soup):
    pass #TODO

In [ ]:
def get_series(soup):
    pass #TODO

In [ ]:
def get_send_time(soup):
    pass

In [ ]:
def get_summary(soup):
    try:
        summary = soup.select('.product-description')[0].text
    except Exception:
        summary = None
        logging.exception("This book has no summary!")
    return summary

In [ ]:
def get_tags(soup):
    pass

In [ ]:
data_list = []
url = "temp"  #TODO
links = get_links(url)

soups = get_soup(links)

for site_soup in soups:
    try:
        book_fa_title = get_fa_title(site_soup)
        book_en_title = get_en_title(site_soup)
        book_price = get_price(site_soup)
        book_discount = get_discount(site_soup)
        book_score = get_score(site_soup)
        book_publisher = get_publisher(site_soup)
        book_author = get_author(site_soup)
        book_code = get_code(site_soup)
        book_Isbn = get_isbn(site_soup)
        book_pages = get_pages(site_soup)
        book_publication_per_date = get_per_date(site_soup)
        book_publication_ad_date = get_ad_date(site_soup)
        book_size = get_size(site_soup)
        book_cover_material = get_material(site_soup)
        book_print_series = get_series(site_soup)
        book_earliest_send_time = get_send_time(site_soup)
        book_summary = get_summary(site_soup)
        book_tags = get_tags(site_soup)

        data = [book_fa_title, book_en_title, book_price, book_discount, book_score, book_publisher, book_author,
                book_code, book_Isbn, book_pages, book_publication_per_date, book_publication_ad_date, book_size,
                book_cover_material, book_print_series, book_earliest_send_time, book_summary, book_tags]  #TODO
        data_list += [data]
    except Exception:
        logging.exception("An error occurred")